In [82]:
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
from astropy.io import fits
import os
from shutil import copyfile

%matplotlib inline

In [33]:
data_dir = '../data/1_17_18/red/'
cal_dir = '../data/1_17_18/cal/'

all_on = glob(data_dir+'*on*.fits')
all_off = glob(data_dir+'*off*.fits')

hi_on = glob(data_dir+'red_hi*on*.fits')
hi_off = glob(data_dir+'red_hi*off*.fits')

med_on = glob(data_dir+'red_med*on*.fits')
med_off = glob(data_dir+'red_med*off*.fits')

low_on = glob(data_dir+'red_low*on*.fits')
low_off = glob(data_dir+'red_low*off*.fits')

flat_on = cal_dir+'master_flat_6.fits'
flat_off = cal_dir+'master_flat_0.fits'

In [19]:
hi_fields = [filename.split('_')[-1].split('.')[0] for filename in hi_on]

med_fields = [filename.split('_')[-1].split('.')[0] for filename in med_on]

low_fields = [filename.split('_')[-1].split('.')[0] for filename in low_on]

In [71]:
flat_on_data = fits.open(flat_on)[0].data
flat_off_data = fits.open(flat_off)[0].data

on_mask = np.zeros(flat_on_data.shape)
off_mask = np.zeros(flat_off_data.shape)

mask_low = 0.7

on_mask[flat_on_data > mask_low] = 1.0
off_mask[flat_off_data > mask_low] = 1.0

In [69]:
def clean_mask(mask_in,offset):
    """
    If a value in the mask is 0, but its surrounded by 1s, set it to one. Does this in place.
    
    Parameters
    ----------
    mask : `~numpy.ndarray`
        mask to fix. Must be 2D array.
    offset : int
        offset to look for ones in
    
    Returns
    -------
    mask : `~numpy.ndarray`
    """
    mask = mask_in.copy()
    
    for i,row in enumerate(mask):
        for j,col in enumerate(mask):
            if (i>=offset-1)&(j>=offset-1)&(i<=mask.shape[0]-offset-1)&(j<=mask.shape[1]-offset-1):
                this_val = mask[i,j]
                left_val = mask[i-offset,j]
                right_val = mask[i+offset,j]
                top_val = off_mask[i,j-offset]
                bottom_val = off_mask[i,j+offset]
                if (this_val == 0.0)&(left_val == 1.0)&(right_val == 1.0)&(top_val == 1.0)&(bottom_val == 1.0):
                    mask[i,j] = 1 
                    
    return mask

In [72]:
on_mask = clean_mask(on_mask,100)
off_mask = clean_mask(off_mask,100)

In [77]:
for im in all_on:
    
    im_hdu = fits.open(im)[0]
    
    im_data = im_hdu.data
    im_header = im_hdu.header

    masked_data = im_data*on_mask
    im_header.set('COMMENT','Data masked with where master flat > {}'.format(mask_low))
    
    out_hdu = fits.PrimaryHDU(masked_data,im_header)
    
    out_name = 'trim_'+im.split('/')[-1]
    
    out_hdu.writeto(out_name,overwrite=True)
    
for im in all_off:
    
    im_hdu = fits.open(im)[0]
    
    im_data = im_hdu.data
    im_header = im_hdu.header

    masked_data = im_data*off_mask
    im_header.set('COMMENT','Data masked with where master flat > {}'.format(mask_low))
    
    out_hdu = fits.PrimaryHDU(masked_data,im_header)
    
    out_name = 'trim_'+data_dir+im.split('/')[-1]
    
    out_hdu.writeto(out_name,overwrite=True)

In [83]:
for field_num in hi_fields:
    os.makedirs('../data/1_17_18/hi_{}'.format(field_num))
    copyfile(data_dir+'trim_red_hi_on_{}.0001.fits'.format(field_num),'../data/1_17_18/hi_{}/trim_hi_on_{}.fits'.format(field_num,field_num))
    copyfile(data_dir+'trim_red_hi_off_{}.0001.fits'.format(field_num),'../data/1_17_18/hi_{}/trim_hi_off_{}.fits'.format(field_num,field_num))
    copyfile(data_dir+'red_hi_on_{}.0001.fits'.format(field_num),'../data/1_17_18/hi_{}/hi_on_{}.fits'.format(field_num,field_num))
    copyfile(data_dir+'red_hi_off_{}.0001.fits'.format(field_num),'../data/1_17_18/hi_{}/hi_off_{}.fits'.format(field_num,field_num))

In [84]:
for field_num in med_fields:
    os.makedirs('../data/1_17_18/med_{}'.format(field_num))
    copyfile(data_dir+'trim_red_med_on_{}.0001.fits'.format(field_num),'../data/1_17_18/med_{}/trim_med_on_{}.fits'.format(field_num,field_num))
    copyfile(data_dir+'trim_red_med_off_{}.0001.fits'.format(field_num),'../data/1_17_18/med_{}/trim_med_off_{}.fits'.format(field_num,field_num))
    copyfile(data_dir+'red_med_on_{}.0001.fits'.format(field_num),'../data/1_17_18/med_{}/med_on_{}.fits'.format(field_num,field_num))
    copyfile(data_dir+'red_med_off_{}.0001.fits'.format(field_num),'../data/1_17_18/med_{}/med_off_{}.fits'.format(field_num,field_num))

In [85]:
for field_num in low_fields:
    os.makedirs('../data/1_17_18/low_{}'.format(field_num))
    copyfile(data_dir+'trim_red_low_on_{}.0001.fits'.format(field_num),'../data/1_17_18/low_{}/trim_low_on_{}.fits'.format(field_num,field_num))
    copyfile(data_dir+'trim_red_low_off_{}.0001.fits'.format(field_num),'../data/1_17_18/low_{}/trim_low_off_{}.fits'.format(field_num,field_num))
    copyfile(data_dir+'red_low_on_{}.0001.fits'.format(field_num),'../data/1_17_18/low_{}/low_on_{}.fits'.format(field_num,field_num))
    copyfile(data_dir+'red_low_off_{}.0001.fits'.format(field_num),'../data/1_17_18/low_{}/low_off_{}.fits'.format(field_num,field_num))

In [90]:
stan_on = glob(data_dir+'*red_bd52_on*')
stan_off = glob(data_dir+'*red_bd52_off*')

for file in stan_on:
    file_end = file.split('/')[-1]
    copyfile(file,'../data/1_17_18/bd52/'+file_end)
for file in stan_on:
    file_end = file.split('/')[-1]
    copyfile(file,'../data/1_17_18/bd52/'+file_end)